## Importing Libraries

In [1]:
!pip install nltk
!pip install spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


## Loading the dataset

In [21]:
full_data = pd.read_csv('/content/1429_1.csv', nrows=5000)
full_data.head()

,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


In [23]:
full_data.isnull().sum()
df.fillna('', inplace=True)

In [4]:
full_data.columns

Index(['id', 'name', 'asins', 'brand', 'categories', 'keys', 'manufacturer',
       'reviews.date', 'reviews.dateAdded', 'reviews.dateSeen',
       'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id',
       'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs',
       'reviews.text', 'reviews.title', 'reviews.userCity',
       'reviews.userProvince', 'reviews.username'],
      dtype='object')

In [24]:
df = full_data[["reviews.text"]]
df["reviews.text"] = df["reviews.text"].astype(str)
full_data.head()

<ipython-input-24-4c639fe51279>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reviews.text"] = df["reviews.text"].astype(str)


,id,name,asins,brand,categories,keys,manufacturer,reviews.date,reviews.dateAdded,reviews.dateSeen,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username
0,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,This product so far has not disappointed. My c...,Kindle,NaN,NaN,Adapter
1,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,great for beginner or experienced person. Boug...,very fast,NaN,NaN,truman
2,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,Inexpensive tablet for him to use and learn on...,Beginner tablet for our 9 year old son.,NaN,NaN,DaveZ
3,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-13T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,4.0,http://reviews.bestbuy.com/3545/5620406/review...,I've had my Fire HD 8 two weeks now and I love...,Good!!!,NaN,NaN,Shacks
4,AVqkIhwDv8e3D1O-lebb,"All-New Fire HD 8 Tablet, 8 HD Display, Wi-Fi,...",B01AHB9CN2,Amazon,"Electronics,iPad & Tablets,All Tablets,Fire Ta...","841667104676,amazon/53004484,amazon/b01ahb9cn2...",Amazon,2017-01-12T00:00:00.000Z,2017-07-03T23:33:15Z,"2017-06-07T09:04:00.000Z,2017-04-30T00:45:00.000Z",...,True,NaN,0.0,5.0,http://reviews.bestbuy.com/3545/5620406/review...,I bought this for my grand daughter when she c...,Fantastic Tablet for kids,NaN,NaN,explore42


## Lower Casing

In [25]:
df["reviews.text_lower"] = df["reviews.text"].str.lower()
df.head()

<ipython-input-25-c050cec1538f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["reviews.text_lower"] = df["reviews.text"].str.lower()


,reviews.text,reviews.text_lower
0,This product so far has not disappointed. My c...,this product so far has not disappointed. my c...
1,great for beginner or experienced person. Boug...,great for beginner or experienced person. boug...
2,Inexpensive tablet for him to use and learn on...,inexpensive tablet for him to use and learn on...
3,I've had my Fire HD 8 two weeks now and I love...,i've had my fire hd 8 two weeks now and i love...
4,I bought this for my grand daughter when she c...,i bought this for my grand daughter when she c...


## Removing Punctuations

In [26]:
#df.drop(["reviews.text_lower"], axis=1, inplace=True)

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["text_wo_punct"] = df["reviews.text_lower"].apply(lambda text: remove_punctuation(text))
df.head()

<ipython-input-26-330b688f61c4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_wo_punct"] = df["reviews.text_lower"].apply(lambda text: remove_punctuation(text))


,reviews.text,reviews.text_lower,text_wo_punct
0,This product so far has not disappointed. My c...,this product so far has not disappointed. my c...,this product so far has not disappointed my ch...
1,great for beginner or experienced person. Boug...,great for beginner or experienced person. boug...,great for beginner or experienced person bough...
2,Inexpensive tablet for him to use and learn on...,inexpensive tablet for him to use and learn on...,inexpensive tablet for him to use and learn on...
3,I've had my Fire HD 8 two weeks now and I love...,i've had my fire hd 8 two weeks now and i love...,ive had my fire hd 8 two weeks now and i love ...
4,I bought this for my grand daughter when she c...,i bought this for my grand daughter when she c...,i bought this for my grand daughter when she c...


## Removing stopwords

In [27]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [29]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

df["text_wo_stop"] = df["text_wo_punct"].apply(lambda text: remove_stopwords(text))
df.head()

<ipython-input-29-b6fef9350b19>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_wo_stop"] = df["text_wo_punct"].apply(lambda text: remove_stopwords(text))


,reviews.text,reviews.text_lower,text_wo_punct,text_wo_stop
0,This product so far has not disappointed. My c...,this product so far has not disappointed. my c...,this product so far has not disappointed my ch...,product far disappointed children love use lik...
1,great for beginner or experienced person. Boug...,great for beginner or experienced person. boug...,great for beginner or experienced person bough...,great beginner experienced person bought gift ...
2,Inexpensive tablet for him to use and learn on...,inexpensive tablet for him to use and learn on...,inexpensive tablet for him to use and learn on...,inexpensive tablet use learn step nabi thrille...
3,I've had my Fire HD 8 two weeks now and I love...,i've had my fire hd 8 two weeks now and i love...,ive had my fire hd 8 two weeks now and i love ...,ive fire hd 8 two weeks love tablet great valu...
4,I bought this for my grand daughter when she c...,i bought this for my grand daughter when she c...,i bought this for my grand daughter when she c...,bought grand daughter comes visit set user ent...


## Removing Frequent words

In [30]:
from collections import Counter
cnt = Counter()
for text in df["text_wo_stop"].values:
    for word in text.split():
        cnt[word] += 1

cnt.most_common(10)

[('tablet', 2102),
 ('great', 1797),
 ('kindle', 1163),
 ('use', 1130),
 ('price', 960),
 ('good', 907),
 ('easy', 822),
 ('bought', 814),
 ('fire', 782),
 ('love', 751)]

In [31]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

df["text_wo_stopfreq"] = df["text_wo_stop"].apply(lambda text: remove_freqwords(text))
df.head()

<ipython-input-31-0ba26e427506>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_wo_stopfreq"] = df["text_wo_stop"].apply(lambda text: remove_freqwords(text))


,reviews.text,reviews.text_lower,text_wo_punct,text_wo_stop,text_wo_stopfreq
0,This product so far has not disappointed. My c...,this product so far has not disappointed. my c...,this product so far has not disappointed my ch...,product far disappointed children love use lik...,product far disappointed children like ability...
1,great for beginner or experienced person. Boug...,great for beginner or experienced person. boug...,great for beginner or experienced person bough...,great beginner experienced person bought gift ...,beginner experienced person gift loves
2,Inexpensive tablet for him to use and learn on...,inexpensive tablet for him to use and learn on...,inexpensive tablet for him to use and learn on...,inexpensive tablet use learn step nabi thrille...,inexpensive learn step nabi thrilled learn sky...
3,I've had my Fire HD 8 two weeks now and I love...,i've had my fire hd 8 two weeks now and i love...,ive had my fire hd 8 two weeks now and i love ...,ive fire hd 8 two weeks love tablet great valu...,ive hd 8 two weeks valuewe prime members shine...
4,I bought this for my grand daughter when she c...,i bought this for my grand daughter when she c...,i bought this for my grand daughter when she c...,bought grand daughter comes visit set user ent...,grand daughter comes visit set user entered ag...


## Removing Rare words

In [32]:
df.drop(["reviews.text_lower","text_wo_punct", "text_wo_stop"], axis=1, inplace=True)

n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

df["text_wo_stopfreqrare"] = df["text_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
df.head()

<ipython-input-32-af1b3fd6fb9b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(["reviews.text_lower","text_wo_punct", "text_wo_stop"], axis=1, inplace=True)
<ipython-input-32-af1b3fd6fb9b>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_wo_stopfreqrare"] = df["text_wo_stopfreq"].apply(lambda text: remove_rarewords(text))


,reviews.text,text_wo_stopfreq,text_wo_stopfreqrare
0,This product so far has not disappointed. My c...,product far disappointed children like ability...,product far disappointed children like ability...
1,great for beginner or experienced person. Boug...,beginner experienced person gift loves,beginner experienced person gift loves
2,Inexpensive tablet for him to use and learn on...,inexpensive learn step nabi thrilled learn sky...,inexpensive learn step nabi thrilled learn sky...
3,I've had my Fire HD 8 two weeks now and I love...,ive hd 8 two weeks valuewe prime members shine...,ive hd 8 two weeks valuewe prime members shine...
4,I bought this for my grand daughter when she c...,grand daughter comes visit set user entered ag...,grand daughter comes visit set user entered ag...


## Tokenization

In [33]:
df["text_wo_stopfreqrare"].unique()

array(['product far disappointed children like ability monitor control content see ease',
       'beginner experienced person gift loves',
       'inexpensive learn step nabi thrilled learn skype already', ...,
       'kids screen isnt strong charging port poorly built',
       'product different color choices wonderful',
       'really fast worth every penny'], dtype=object)

In [34]:
index = 2
col = "text_wo_stopfreqrare"
cell_val = df.iloc[index][col]
#print "Cell value at ", index, "for column ", col, " : ", cell_val
text=cell_val


#text = "This is an example sentence."
tokens = word_tokenize(text)
print(tokens)

['inexpensive', 'learn', 'step', 'nabi', 'thrilled', 'learn', 'skype', 'already']


## Lemmatization

In [47]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [48]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df["text_lemmatized"] = df["reviews.text"].apply(lambda text: lemmatize_words(text))
df.head()

<ipython-input-48-0372685505e8>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_lemmatized"] = df["reviews.text"].apply(lambda text: lemmatize_words(text))


,reviews.text,text_wo_stopfreq,text_wo_stopfreqrare,text_lemmatized
0,This product so far has not disappointed. My c...,product far disappointed children like ability...,product far disappointed children like ability...,This product so far ha not disappointed. My ch...
1,great for beginner or experienced person. Boug...,beginner experienced person gift loves,beginner experienced person gift loves,great for beginner or experienced person. Boug...
2,Inexpensive tablet for him to use and learn on...,inexpensive learn step nabi thrilled learn sky...,inexpensive learn step nabi thrilled learn sky...,Inexpensive tablet for him to use and learn on...
3,I've had my Fire HD 8 two weeks now and I love...,ive hd 8 two weeks valuewe prime members shine...,ive hd 8 two weeks valuewe prime members shine...,I've had my Fire HD 8 two week now and I love ...
4,I bought this for my grand daughter when she c...,grand daughter comes visit set user entered ag...,grand daughter comes visit set user entered ag...,I bought this for my grand daughter when she c...


In [49]:
from nltk.stem import PorterStemmer

# Load the English language model in spaCy
nlp = spacy.load('en_core_web_sm')

# Extract lemmas using spaCy
index = 2
col = "text_lemmatized"
cell_val = str(df.iloc[index][col])
lemmas = [token.lemma_ for token in cell_val]
print(lemmas)

# Define tokens from the processed Doc object
tokens = [token.text for token in df["text_wo_stopfreqrare"]]

# Create an instance of PorterStemmer from NLTK
stemmer = PorterStemmer()

# Apply stemming on the tokens using PorterStemmer
stems = [stemmer.stem(word) for word in tokens]
print(stems)

AttributeError: ignored